In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
train_beneficiary = pd.read_csv('Train_Beneficiarydata-1542865627584.csv')
train_inpatient = pd.read_csv('Train_Inpatientdata-1542865627584.csv')
train_outpatient = pd.read_csv('Train_Outpatientdata-1542865627584.csv')


In [3]:

train_beneficiary.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138556 entries, 0 to 138555
Data columns (total 25 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   BeneID                           138556 non-null  object
 1   DOB                              138556 non-null  object
 2   DOD                              1421 non-null    object
 3   Gender                           138556 non-null  int64 
 4   Race                             138556 non-null  int64 
 5   RenalDiseaseIndicator            138556 non-null  object
 6   State                            138556 non-null  int64 
 7   County                           138556 non-null  int64 
 8   NoOfMonths_PartACov              138556 non-null  int64 
 9   NoOfMonths_PartBCov              138556 non-null  int64 
 10  ChronicCond_Alzheimer            138556 non-null  int64 
 11  ChronicCond_Heartfailure         138556 non-null  int64 
 12  ChronicCond_Kidn

In [4]:
train_beneficiary.head()


,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,...,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NaN,1,1,0,39,230,12,12,...,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NaN,2,1,0,39,280,12,12,...,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NaN,1,1,0,52,590,12,12,...,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NaN,1,1,0,39,270,12,12,...,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NaN,1,1,0,24,680,12,12,...,2,1,2,2,2,2,0,0,1790,1200


In [5]:
train_beneficiary.shape

(138556, 25)

In [6]:
#Adjust date based columns to datetime format
train_beneficiary['DOB'] = pd.to_datetime(train_beneficiary['DOB'])
train_inpatient['AdmissionDt'] = pd.to_datetime(train_inpatient['AdmissionDt'])
train_inpatient['DischargeDt'] = pd.to_datetime(train_inpatient['DischargeDt'])
train_outpatient['ClaimStartDt'] = pd.to_datetime(train_outpatient['ClaimStartDt'])
train_outpatient['ClaimEndDt'] = pd.to_datetime(train_outpatient['ClaimEndDt'])


In [7]:
#Scrub for NAN values
train_beneficiary.isnull().sum()

BeneID                                  0
DOB                                     0
DOD                                137135
Gender                                  0
Race                                    0
RenalDiseaseIndicator                   0
State                                   0
County                                  0
NoOfMonths_PartACov                     0
NoOfMonths_PartBCov                     0
ChronicCond_Alzheimer                   0
ChronicCond_Heartfailure                0
ChronicCond_KidneyDisease               0
ChronicCond_Cancer                      0
ChronicCond_ObstrPulmonary              0
ChronicCond_Depression                  0
ChronicCond_Diabetes                    0
ChronicCond_IschemicHeart               0
ChronicCond_Osteoporasis                0
ChronicCond_rheumatoidarthritis         0
ChronicCond_stroke                      0
IPAnnualReimbursementAmt                0
IPAnnualDeductibleAmt                   0
OPAnnualReimbursementAmt          

In [8]:
inpatient_unique_values = train_beneficiary['DOD'].unique()
print(inpatient_unique_values)


[nan '2009-12-01' '2009-09-01' '2009-05-01' '2009-07-01' '2009-08-01'
 '2009-10-01' '2009-06-01' '2009-04-01' '2009-02-01' '2009-11-01'
 '2009-03-01']


In [9]:


# Replace NaN values with a future date, such as '2099-12-31'
train_beneficiary['DOD'] = train_beneficiary['DOD'].fillna('2099-12-31')


In [10]:
train_beneficiary.isnull().sum()

BeneID                             0
DOB                                0
DOD                                0
Gender                             0
Race                               0
RenalDiseaseIndicator              0
State                              0
County                             0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
dtype: int64

In [11]:
# Check for duplicates
train_beneficiary.duplicated().sum()

0

In [12]:
# Begin cleaning the train_inpatient dataset
train_inpatient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40474 entries, 0 to 40473
Data columns (total 30 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   BeneID                  40474 non-null  object        
 1   ClaimID                 40474 non-null  object        
 2   ClaimStartDt            40474 non-null  object        
 3   ClaimEndDt              40474 non-null  object        
 4   Provider                40474 non-null  object        
 5   InscClaimAmtReimbursed  40474 non-null  int64         
 6   AttendingPhysician      40362 non-null  object        
 7   OperatingPhysician      23830 non-null  object        
 8   OtherPhysician          4690 non-null   object        
 9   AdmissionDt             40474 non-null  datetime64[ns]
 10  ClmAdmitDiagnosisCode   40474 non-null  object        
 11  DeductibleAmtPaid       39575 non-null  float64       
 12  DischargeDt             40474 non-null  dateti

In [13]:
train_inpatient.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,AdmissionDt,...,ClmDiagnosisCode_7,ClmDiagnosisCode_8,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6
0,BENE11001,CLM46614,2009-04-12,2009-04-18,PRV55912,26000,PHY390922,NaN,NaN,2009-04-12,...,2724,19889,5849,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BENE11001,CLM66048,2009-08-31,2009-09-02,PRV55907,5000,PHY318495,PHY318495,NaN,2009-08-31,...,NaN,NaN,NaN,NaN,7092.0,NaN,NaN,NaN,NaN,NaN
2,BENE11001,CLM68358,2009-09-17,2009-09-20,PRV56046,5000,PHY372395,NaN,PHY324689,2009-09-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BENE11011,CLM38412,2009-02-14,2009-02-22,PRV52405,5000,PHY369659,PHY392961,PHY349768,2009-02-14,...,25062,40390,4019,NaN,331.0,NaN,NaN,NaN,NaN,NaN
4,BENE11014,CLM63689,2009-08-13,2009-08-30,PRV56614,10000,PHY379376,PHY398258,NaN,2009-08-13,...,5119,29620,20300,NaN,3893.0,NaN,NaN,NaN,NaN,NaN


In [14]:
train_inpatient.isnull().sum()

BeneID                        0
ClaimID                       0
ClaimStartDt                  0
ClaimEndDt                    0
Provider                      0
InscClaimAmtReimbursed        0
AttendingPhysician          112
OperatingPhysician        16644
OtherPhysician            35784
AdmissionDt                   0
ClmAdmitDiagnosisCode         0
DeductibleAmtPaid           899
DischargeDt                   0
DiagnosisGroupCode            0
ClmDiagnosisCode_1            0
ClmDiagnosisCode_2          226
ClmDiagnosisCode_3          676
ClmDiagnosisCode_4         1534
ClmDiagnosisCode_5         2894
ClmDiagnosisCode_6         4838
ClmDiagnosisCode_7         7258
ClmDiagnosisCode_8         9942
ClmDiagnosisCode_9        13497
ClmDiagnosisCode_10       36547
ClmProcedureCode_1        17326
ClmProcedureCode_2        35020
ClmProcedureCode_3        39509
ClmProcedureCode_4        40358
ClmProcedureCode_5        40465
ClmProcedureCode_6        40474
dtype: int64

In [15]:
# Fill missing values for claim columns with 0
train_inpatient['ClmAdmitDiagnosisCode'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_1'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_2'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_3'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_4'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_5'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_6'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_7'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_8'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_9'].fillna(0, inplace=True)
train_inpatient['ClmDiagnosisCode_10'].fillna(0, inplace=True)
train_inpatient['ClmProcedureCode_1'].fillna(0, inplace=True)
train_inpatient['ClmProcedureCode_2'].fillna(0, inplace=True)
train_inpatient['ClmProcedureCode_3'].fillna(0, inplace=True)
train_inpatient['ClmProcedureCode_4'].fillna(0, inplace=True)
train_inpatient['ClmProcedureCode_5'].fillna(0, inplace=True)
train_inpatient['ClmProcedureCode_6'].fillna(0, inplace=True)

# Fill missing values for provider columns with Unknown
train_inpatient['AttendingPhysician'].fillna('Unknown', inplace=True)
train_inpatient['OperatingPhysician'].fillna('Unknown', inplace=True)
train_inpatient['OtherPhysician'].fillna('Unknown', inplace=True)
train_inpatient['ClmAdmitDiagnosisCode'].fillna('Unknown', inplace=True)

#  Fill missing values for DeductibleAmtPaid with 0
train_inpatient['DeductibleAmtPaid'].fillna(0, inplace=True)

train_inpatient.isnull().sum()



BeneID                    0
ClaimID                   0
ClaimStartDt              0
ClaimEndDt                0
Provider                  0
InscClaimAmtReimbursed    0
AttendingPhysician        0
OperatingPhysician        0
OtherPhysician            0
AdmissionDt               0
ClmAdmitDiagnosisCode     0
DeductibleAmtPaid         0
DischargeDt               0
DiagnosisGroupCode        0
ClmDiagnosisCode_1        0
ClmDiagnosisCode_2        0
ClmDiagnosisCode_3        0
ClmDiagnosisCode_4        0
ClmDiagnosisCode_5        0
ClmDiagnosisCode_6        0
ClmDiagnosisCode_7        0
ClmDiagnosisCode_8        0
ClmDiagnosisCode_9        0
ClmDiagnosisCode_10       0
ClmProcedureCode_1        0
ClmProcedureCode_2        0
ClmProcedureCode_3        0
ClmProcedureCode_4        0
ClmProcedureCode_5        0
ClmProcedureCode_6        0
dtype: int64

In [16]:
# Clean up the train_outpatient dataset
train_outpatient.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517737 entries, 0 to 517736
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   BeneID                  517737 non-null  object        
 1   ClaimID                 517737 non-null  object        
 2   ClaimStartDt            517737 non-null  datetime64[ns]
 3   ClaimEndDt              517737 non-null  datetime64[ns]
 4   Provider                517737 non-null  object        
 5   InscClaimAmtReimbursed  517737 non-null  int64         
 6   AttendingPhysician      516341 non-null  object        
 7   OperatingPhysician      90617 non-null   object        
 8   OtherPhysician          195046 non-null  object        
 9   ClmDiagnosisCode_1      507284 non-null  object        
 10  ClmDiagnosisCode_2      322357 non-null  object        
 11  ClmDiagnosisCode_3      203257 non-null  object        
 12  ClmDiagnosisCode_4      125596

In [17]:
train_outpatient.isnull().sum()

BeneID                         0
ClaimID                        0
ClaimStartDt                   0
ClaimEndDt                     0
Provider                       0
InscClaimAmtReimbursed         0
AttendingPhysician          1396
OperatingPhysician        427120
OtherPhysician            322691
ClmDiagnosisCode_1         10453
ClmDiagnosisCode_2        195380
ClmDiagnosisCode_3        314480
ClmDiagnosisCode_4        392141
ClmDiagnosisCode_5        443393
ClmDiagnosisCode_6        468981
ClmDiagnosisCode_7        484776
ClmDiagnosisCode_8        494825
ClmDiagnosisCode_9        502899
ClmDiagnosisCode_10       516654
ClmProcedureCode_1        517575
ClmProcedureCode_2        517701
ClmProcedureCode_3        517733
ClmProcedureCode_4        517735
ClmProcedureCode_5        517737
ClmProcedureCode_6        517737
DeductibleAmtPaid              0
ClmAdmitDiagnosisCode     412312
dtype: int64

In [18]:
train_outpatient.head()

,BeneID,ClaimID,ClaimStartDt,ClaimEndDt,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,...,ClmDiagnosisCode_9,ClmDiagnosisCode_10,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,ClmProcedureCode_4,ClmProcedureCode_5,ClmProcedureCode_6,DeductibleAmtPaid,ClmAdmitDiagnosisCode
0,BENE11002,CLM624349,2009-10-11,2009-10-11,PRV56011,30,PHY326117,NaN,NaN,78943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,56409
1,BENE11003,CLM189947,2009-02-12,2009-02-12,PRV57610,80,PHY362868,NaN,NaN,6115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,79380
2,BENE11003,CLM438021,2009-06-27,2009-06-27,PRV57595,10,PHY328821,NaN,NaN,2723,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
3,BENE11004,CLM121801,2009-01-06,2009-01-06,PRV56011,40,PHY334319,NaN,NaN,71988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,BENE11004,CLM150998,2009-01-22,2009-01-22,PRV56011,200,PHY403831,NaN,NaN,82382,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,71947


In [19]:
# Replace duplicate diagnosis and procedure data
train_outpatient ['ClmDiagnosisCode_1'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_2'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_3'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_4'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_5'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_6'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_7'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_8'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_9'].fillna(0, inplace=True)
train_outpatient ['ClmDiagnosisCode_10'].fillna(0, inplace=True)
train_outpatient ['ClmProcedureCode_1'].fillna(0, inplace=True)
train_outpatient ['ClmProcedureCode_2'].fillna(0, inplace=True)
train_outpatient ['ClmProcedureCode_3'].fillna(0, inplace=True)
train_outpatient ['ClmProcedureCode_4'].fillna(0, inplace=True)
train_outpatient ['ClmProcedureCode_5'].fillna(0, inplace=True)
train_outpatient ['ClmProcedureCode_6'].fillna(0, inplace=True)

#Replace ClaimAdmitDiagnosisCode with Unknown
train_outpatient ['ClmAdmitDiagnosisCode'].fillna('Unknown', inplace=True)

# Replace AttendingPhysician, OperatingPhysician, OtherPhysician with Unknown
train_outpatient ['AttendingPhysician'].fillna('Unknown', inplace=True)
train_outpatient ['OperatingPhysician'].fillna('Unknown', inplace=True)
train_outpatient ['OtherPhysician'].fillna('Unknown', inplace=True)

#Check for nulls
train_outpatient.isnull().sum()

BeneID                    0
ClaimID                   0
ClaimStartDt              0
ClaimEndDt                0
Provider                  0
InscClaimAmtReimbursed    0
AttendingPhysician        0
OperatingPhysician        0
OtherPhysician            0
ClmDiagnosisCode_1        0
ClmDiagnosisCode_2        0
ClmDiagnosisCode_3        0
ClmDiagnosisCode_4        0
ClmDiagnosisCode_5        0
ClmDiagnosisCode_6        0
ClmDiagnosisCode_7        0
ClmDiagnosisCode_8        0
ClmDiagnosisCode_9        0
ClmDiagnosisCode_10       0
ClmProcedureCode_1        0
ClmProcedureCode_2        0
ClmProcedureCode_3        0
ClmProcedureCode_4        0
ClmProcedureCode_5        0
ClmProcedureCode_6        0
DeductibleAmtPaid         0
ClmAdmitDiagnosisCode     0
dtype: int64

In [20]:
#Remove duplicates
train_outpatient.duplicated().sum()

0

In [21]:
# Determine the shape of the train_outpatient dataset
train_outpatient.shape

(517737, 27)

In [22]:
# Determine the shape of the inpatient dataset
train_inpatient.shape



(40474, 30)

In [23]:
# Determine the shape of the beneficiary dataset
train_beneficiary.shape


(138556, 25)

In [24]:

# Creating flags in inpatient and outpatient data sets
train_inpatient['IsInpatient'] = 1
train_outpatient['IsInpatient'] = 0


In [25]:
# Concat both train inpatient and outpatient
train_claims = pd.concat([train_inpatient, train_outpatient], axis=0, ignore_index=True)


In [26]:
train_claims.shape

(558211, 31)

In [27]:
train_claims.columns

Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'IsInpatient'],
      dtype='object')

In [28]:

train_data = pd.merge(train_claims, train_beneficiary, on='BeneID', how='left')
train_data.columns


Index(['BeneID', 'ClaimID', 'ClaimStartDt', 'ClaimEndDt', 'Provider',
       'InscClaimAmtReimbursed', 'AttendingPhysician', 'OperatingPhysician',
       'OtherPhysician', 'AdmissionDt', 'ClmAdmitDiagnosisCode',
       'DeductibleAmtPaid', 'DischargeDt', 'DiagnosisGroupCode',
       'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
       'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
       'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
       'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
       'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
       'ClmProcedureCode_6', 'IsInpatient', 'DOB', 'DOD', 'Gender', 'Race',
       'RenalDiseaseIndicator', 'State', 'County', 'NoOfMonths_PartACov',
       'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depressio

In [29]:
train_data['LengthOfStay'] = (pd.to_datetime(train_data['DischargeDt']) - pd.to_datetime(train_data['AdmissionDt'])).dt.days
train_data['NumChronicCond'] = train_data[['ChronicCond_Alzheimer', 'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease', 'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']].sum(axis=1)


In [30]:
train_data.isnull().sum()

BeneID                                  0
ClaimID                                 0
ClaimStartDt                            0
ClaimEndDt                              0
Provider                                0
InscClaimAmtReimbursed                  0
AttendingPhysician                      0
OperatingPhysician                      0
OtherPhysician                          0
AdmissionDt                        517737
ClmAdmitDiagnosisCode                   0
DeductibleAmtPaid                       0
DischargeDt                        517737
DiagnosisGroupCode                 517737
ClmDiagnosisCode_1                      0
ClmDiagnosisCode_2                      0
ClmDiagnosisCode_3                      0
ClmDiagnosisCode_4                      0
ClmDiagnosisCode_5                      0
ClmDiagnosisCode_6                      0
ClmDiagnosisCode_7                      0
ClmDiagnosisCode_8                      0
ClmDiagnosisCode_9                      0
ClmDiagnosisCode_10               

In [31]:
# Fill LengthOfStay null values with 0 to represent outpatients as not apart of that column.
train_data['LengthOfStay']= train_data['LengthOfStay'].fillna(0)


In [32]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 558210
Data columns (total 57 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   BeneID                           558211 non-null  object        
 1   ClaimID                          558211 non-null  object        
 2   ClaimStartDt                     558211 non-null  object        
 3   ClaimEndDt                       558211 non-null  object        
 4   Provider                         558211 non-null  object        
 5   InscClaimAmtReimbursed           558211 non-null  int64         
 6   AttendingPhysician               558211 non-null  object        
 7   OperatingPhysician               558211 non-null  object        
 8   OtherPhysician                   558211 non-null  object        
 9   AdmissionDt                      40474 non-null   datetime64[ns]
 10  ClmAdmitDiagnosisCode            558211 non-

In [33]:
fraud_labels = pd.read_csv('Train-1542865627584.csv')

# Merge the fraud_labels with train_data based on the 'Provider' column
train_data = pd.merge(train_data, fraud_labels, on='Provider')


In [34]:
train_data = train_data.rename(columns={'PotentialFraud': 'Fraud'})


In [35]:
train_data['Fraud'] = train_data['Fraud'].map({'Yes': 1, 'No': 0})


In [36]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 558211 entries, 0 to 558210
Data columns (total 58 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   BeneID                           558211 non-null  object        
 1   ClaimID                          558211 non-null  object        
 2   ClaimStartDt                     558211 non-null  object        
 3   ClaimEndDt                       558211 non-null  object        
 4   Provider                         558211 non-null  object        
 5   InscClaimAmtReimbursed           558211 non-null  int64         
 6   AttendingPhysician               558211 non-null  object        
 7   OperatingPhysician               558211 non-null  object        
 8   OtherPhysician                   558211 non-null  object        
 9   AdmissionDt                      40474 non-null   datetime64[ns]
 10  ClmAdmitDiagnosisCode            558211 non-

In [37]:
# Remove DOD column
train_data = train_data.drop(['DOD'], axis=1)


In [38]:
# Remove columns BeneID and ClaimID
train_data = train_data.drop(['BeneID', 'ClaimID'], axis=1)


In [39]:
# Create ClaimDuration column from 'ClaimStartDt' and 'ClaimEndDt' to represent the duration of a claim
train_data['ClaimDuration'] = (pd.to_datetime(train_data['ClaimEndDt']) - pd.to_datetime(train_data['ClaimStartDt'])).dt.days


In [40]:
def count_unique_physicians(row):
    physicians = set([row['AttendingPhysician'], row['OperatingPhysician'], row['OtherPhysician']])
    physicians.discard(np.nan)
    return len(physicians)

train_data['NumUniquePhysicians'] = train_data.apply(count_unique_physicians, axis=1)


In [41]:
# Create a new column from 'ClaimStartDt' and 'DOB'  to identify the patients age at the time of the claim
train_data['PatientAge'] = (pd.to_datetime(train_data['ClaimStartDt']) - pd.to_datetime(train_data['DOB'])).dt.days / 365.25



In [42]:
train_data['PatientAge'] = train_data['PatientAge'].astype(int)
train_data['PatientAge']


0         66
1         95
2         87
3         79
4         83
          ..
558206    81
558207    67
558208    63
558209    84
558210    66
Name: PatientAge, Length: 558211, dtype: int64

In [43]:
# Remove 'AdmissionDt' and 'DischargeDt' columns
train_data = train_data.drop(['AdmissionDt', 'DischargeDt'], axis=1)

# Split the dataset into features (X) and target (y)
X = train_data.drop('Fraud', axis=1)
y = train_data['Fraud']

# Define the categorical and numerical columns
categorical_columns = [
    'Provider', 'AttendingPhysician', 'OperatingPhysician', 'OtherPhysician',
    'ClmAdmitDiagnosisCode', 'DiagnosisGroupCode',
    'ClmDiagnosisCode_1', 'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3',
    'ClmDiagnosisCode_4', 'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6',
    'ClmDiagnosisCode_7', 'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9',
    'ClmDiagnosisCode_10', 'ClmProcedureCode_1', 'ClmProcedureCode_2',
    'ClmProcedureCode_3', 'ClmProcedureCode_4', 'ClmProcedureCode_5',
    'ClmProcedureCode_6', 'IsInpatient', 'Gender', 'Race',
    'RenalDiseaseIndicator', 'State', 'County'
]

numerical_columns = [
    'InscClaimAmtReimbursed', 'DeductibleAmtPaid',
    'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
    'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
    'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
    'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
    'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
    'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
    'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
    'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
    'OPAnnualDeductibleAmt', 'LengthOfStay', 'NumChronicCond',
    'ClaimDuration', 'NumUniquePhysicians', 'PatientAge'
]




In [44]:


# Number of components to keep
n_components = 50

# Create a column transformer for the categorical columns with OneHotEncoder and PCA
cat_preprocessor = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore')),
                                   ('pca', TruncatedSVD(n_components=n_components))])


# Combine the results with the numerical columns using FeatureUnion
preprocessor = FeatureUnion(transformer_list=[
    ('num', ColumnTransformer(transformers=[('num', StandardScaler(), numerical_columns)])),
    ('cat', ColumnTransformer(transformers=[('cat', cat_preprocessor, categorical_columns)]))
])

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert all categorical columns to string data type for both X_train and X_test
for column in categorical_columns:
    X_train[column] = X_train[column].astype(str)
    X_test[column] = X_test[column].astype(str)

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create a pipeline with the preprocessor and the Random Forest classifier
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', RandomForestClassifier(random_state=42))])

# Fit the pipeline on the training data
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Print the classification report
print(classification_report(y_test,  y_pred))

# Print the confusion matrix
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.96      0.90     69201
           1       0.91      0.74      0.82     42442

    accuracy                           0.87    111643
   macro avg       0.88      0.85      0.86    111643
weighted avg       0.88      0.87      0.87    111643

[[66199  3002]
 [11069 31373]]
